DETECTING FINGERS :

1.Defining an ROI where fingers will be detected.

2.Running background analysis(Getting average background value) to study the background so that elements in the backgrounds would not be mistook as fingers.

3.Finding the largest contour in the ROI assuming it to be the hand.

4.Obtaining the topmost , rightend and leftend (extreme points) and finding the co-ordinates of the center of the hand by using convexHull() method.

5.Finding the euclidean distances between the center and the extreme points and storing the maximum one.

5.Creating a circular ROI with radius being 90% of the maximum euclidean distance and masking it on the thresholded version.

6.All the external contours are then detected in the imaginary circular ROI and contours which lie in the limit_points(circumference of the circular ROI) and out_of_wrist (Height greater than  wrist points) are detected as fingers and no.of finger counters are counted hence giving the no.of fingers.

In [1]:
#Importing all the necessary libraries
import cv2
import numpy as np
from sklearn.metrics import pairwise #For calculating the size of the palm and the fingers


In [2]:
# GLOBAL VARIABLES TO BE USED

bg = None

accum_wt = 0.5 #Default value

#Setting the boundaries for region of interest
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_bg_accum_avg(frame,accum_wt):
    global bg
    if bg is None:
        bg = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,bg,accum_wt)

In [4]:
def segment_hand(frame,threshold = 25):
    global bg
    
    #Calculating absolute difference between frame and backgorund
    diff = cv2.absdiff(bg.astype('uint8'),frame)
    #Applying threshold to grab the foreground 
    
    ret,threshold_img = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    #Getting the external contours
    contours,_ = cv2.findContours(threshold_img.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key = cv2.contourArea)
        
        return (threshold_img,hand_segment)

In [5]:
def counting_fingers(threshold_img,hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    
    #Grabbing the outermost corner points of the convex hull
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    #Getting the co-ordinates center of the palm
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    #Finding the euclidean distance between center of the palm
    #and the extreme points of the convex hull
    
    distance = pairwise.euclidean_distances([(cX,cY)],Y = [left,right,top,bottom])[0]
    
    #Grabbing the largest distance
    max_distance = distance.max()
    
    #Creating a circle with 80 to 90% radius of the largest euclidean distance
    radius = int(0.8 * max_distance)
    circumference = (2*np.pi * radius)
    
    circular_roi = np.zeros(threshold_img.shape[:2],dtype = 'uint8')
    
    #Drawing the circular roi
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    circular_roi = cv2.bitwise_and(threshold_img,threshold_img,mask = circular_roi)
    
    contours,_ = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours :
        
        #Bounding box for contours
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        #Counting the fingers (incrementing the count variable)
        #Based on 2 factors
        
        #1.Contour region is not the wrist area
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
                        
        #2.Points along the contour should not exceed 25% of the circumference of the circualar roi
        #(else we would be counting points of hand)
                        
        limit_boundary_points = ((circumference * 0.25) > cnt.shape[0])

        if out_of_wrist and limit_boundary_points:
                        count+=1
    return count

In [ ]:
#Putting everything together

cap = cv2.VideoCapture(0)

#Initializing the frame count
num_frames = 0

while True:
    #Get the current frame
    ret,frame = cap.read()
    
    if ret == True:
        
        #Flipping the camera to avoid the mirror view
        frame = cv2.flip(frame,1)
        
        #Cloning the frame 
        frame_copy = frame.copy()
        
        #Grab the ROI from the frame
        roi = frame[roi_top : roi_bottom,roi_right:roi_left]
        
        #Apply grayscale and blur to ROI 
        gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray,(7,7),0)
        
        #Letting the user know while the background weights are being averaged 
        
        if num_frames < 60 : 
            calc_bg_accum_avg(gray,accum_wt)
            if num_frames <= 59:
                cv2.putText(frame_copy,"WAIT ! WORK IN PROGRESS",(200,400),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
                cv2.imshow("Finger_count",frame_copy)
                
        else:
            #Segmenting the hand region
            hand = segment_hand(gray)
            
            if hand is not None:
                
                #unpack 
                threhsold_img,hand_segment = hand
                
                #Draw contours arounf hand segment 
                cv2.drawContours(frame_copy,[hand_segment + (roi_right,roi_top)],-1,(255,0,0),1)
                
                #Counting the fingers
                
                fingers = counting_fingers(threshold_img,hand_segment)
                
                #Displaying the count
                cv2.putText(frame_copy,str(finger),(70,45),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
                
                #Displaying the thresholded image
                cv2.imshow("Threshold_img",threshold_img)
            
            #Drawing the ROI
            cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
            
            #increment the number of framees based
            
            
            num_frames += 1
            
            cv2.imshow("Finger_count",frame_copy)
            
            key = cv2.waitKey(1) & 0xff
            
            if key == 27:
                break
    else:
                break
cap.release()
cv2.destroyAllWindows()
            
              

In [38]:
cam = cv2.VideoCapture(0)

# Intialize a frame count
num_frames = 0

# keep looping, until interrupted
while True:
    # get the current frame
    ret, frame = cam.read()
    
    if ret == True:

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        frame_copy = frame.copy()

        # Grab the ROI from the frame
        roi = frame[roi_top:roi_bottom, roi_right:roi_left]

        # Apply grayscale and blur to ROI
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # For the first 30 frames we will calculate the average of the background.
        # We will tell the user while this is happening
        if num_frames < 60:
            calc_bg_accum_avg(gray, accum_wt)
            if num_frames <= 59:
                cv2.putText(frame_copy, "WAIT! WORK IN PROGRESS.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                cv2.imshow("Finger Count",frame_copy)

        else:
            # now that we have the background, we can segment the hand.

            # segment the hand region
            hand = segment_hand(gray)

            # First check if we were able to actually detect a hand
            if hand is not None:

                # unpack
                thresholded, hand_segment = hand

                # Draw contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

                # Count the fingers
                fingers = counting_fingers(thresholded, hand_segment)

                # Display count
                cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

                # Also display the thresholded image
                cv2.imshow("Thesholded", thresholded)

        # Draw ROI Rectangle on frame copy
        cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

        # increment the number of frames for tracking
        num_frames += 1

        # Display the frame with segmented hand
        cv2.imshow("Finger Count", frame_copy)


        # Close windows with Esc
        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            break
    else:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()